In [1]:
#calculating work orders closed on time in facilities maintenance division

 - __Measure Description:__This KPI provides information on  how long it takes to close facility maintenance work orders .  It measures the average work order duration of each  individual work order within in each of the varous work order problem types and captures both the duration of each specific work order as well as the average work order duration categorically for each work order problem type.  Determining which and the total number of individual work orders that exceed the average duration for their respective problem type provides insight on the percentage of work orders closed below and above normal duration.   (This approach allows the department to compare apples to apples with respect to work order workload and duration because of natural variations in the amount of time needed to close a work order that are directly related to the type of work order request.  For example, it will take longer to close a roofing work order than a door knob repair or a custodial work order given the nature of the work and potential for waiting time for parts.)
 - __Data Source:__ Source of this data is the Archibus--the DGS Facility Maintenance work order management software.  Below is an example to illustrate method of calcluation.  Total # of work orders with duration at or below the average duration for its specific work order problem type = TWOOT.  Total # of work orders in fiscal year = TWOFY.   The measure is ((TWOOT/TWOFY)x.01)
 - __Purpose:__ This measure allows us to effectively compare activity on work orders against quarterly, yearly or other periods. This indicator has helped to develop problem type specific and Facility Maintenance shop specific strategies to address strengths, weaknesses and opportunities.  Use of this indicator can support the future development of more granulalar duration standards by category.  It also informs the division on the potential impact that factors such as  parts and equipment delivery may have on customer service perceptions. 

In [2]:
# break up into three sections:
# 1. data analysis to make calculations for work orders closed on time + functions that edit data types/Fiscal Year
# 2. plot files to plot average duration top 10 work orders over a year in bubble chart?
# 3. file that brings all functions together

In [3]:
import pandas as pd
import numpy as np

#packages for updating archibus-connected excel sheet
import win32com.client
from time import sleep

In [4]:
# import data

#function to automatically update archibus connected excel sheet
#and return a df of updated data
#create function to give an updated archibus dataframe
def ArchibusUpdate():
    # Start an instance of Excel
    xlapp = win32com.client.DispatchEx("Excel.Application")
    
    # Open the workbook in said instance of Excel, workbook will refresh on open
    wb = xlapp.workbooks.open("Z:\\BLima\\Archibus Dump 04-27-17-Database connected.xlsx")

    #sleep to allow time for document to refresh
    sleep(30)

    #save the document
    wb.Save()

    #sleep to allow time for document to save
    sleep(30)

    #quit excel
    xlapp.Quit()
    
    #save updated excel sheet as new dataframe = df
    df=pd.read_excel("Z:\\BLima\\Archibus Dump 04-27-17-Database connected.xlsx")
    
    #print dataframe
    return df

In [5]:
df=ArchibusUpdate()

In [6]:
df.columns.tolist()

['rm_id',
 'fl_id',
 'bl_id',
 'site_id',
 'ac_id',
 'act_labor_hours',
 'activity_log_id',
 'activity_type',
 'allow_work_on_holidays',
 'cause_type',
 'cf_notes',
 'completed_by',
 'cost_est_labor',
 'cost_est_other',
 'cost_est_parts',
 'cost_est_tools',
 'cost_est_total',
 'cost_labor',
 'cost_other',
 'cost_parts',
 'cost_tools',
 'cost_total',
 'curr_meter_val',
 'date_assigned',
 'date_closed',
 'date_completed',
 'date_escalation_completion',
 'date_escalation_response',
 'date_est_completion',
 'date_requested',
 'date_stat_chg',
 'desc_other_costs',
 'description',
 'dispatcher',
 'doc1',
 'doc2',
 'doc3',
 'doc4',
 'down_time',
 'dp_id',
 'dv_id',
 'eq_id',
 'escalated_completion',
 'escalated_response',
 'est_labor_hours',
 'location',
 'manager',
 'msg_delivery_status',
 'option1',
 'option2',
 'phone',
 'pmp_id',
 'pms_id',
 'priority',
 'prob_type',
 'repair_type',
 'requestor',
 'res_id',
 'rmres_id',
 'rsres_id',
 'satisfaction',
 'satisfaction_notes',
 'serv_window_da

In [7]:
# create dataframe that  includes only relevant columns, adds month/fiscal year columns for duration-related col

def readable_df():
    df=ArchibusUpdate()
    #list of column names to be used in updated column
    col=['bl_id', 
         'site_id',
         'prob_type',
         'completed_by', 
         'cost_total', 
         'date_assigned',
         'date_closed',
         'date_completed',
         'date_requested',
         'manager',
         'time_assigned',
         'time_completed',
         'time_requested',
         'time_stat_chg',
         'po_number',
         'name']
    
    df=df.filter(items=col)
    
    # add in month and fiscal year columns for dates requested-completed
    df['date_requested_month']=df['date_requested'].dt.month
    df['date_requested_fiscalyear']=np.where(df['date_requested_month']<7, df['date_requested'].dt.year,df['date_requested'].dt.year+1)
    
    df['date_completed_month']=df['date_completed'].dt.month
    df['date_completed_fiscalyear']=np.where(df['date_completed_month']<7, df['date_completed'].dt.year,df['date_completed'].dt.year+1)
    return df
    

In [8]:
df=readable_df()

In [9]:
df.head()

,bl_id,site_id,prob_type,completed_by,cost_total,date_assigned,date_closed,date_completed,date_requested,manager,time_assigned,time_completed,time_requested,time_stat_chg,po_number,name,date_requested_month,date_requested_fiscalyear,date_completed_month,date_completed_fiscalyear
0,B00063,DGS,PREVENTIVE MAINT,NaN,0.0,NaT,NaT,NaT,2018-08-02,KAGESTA.CUPID,NaN,NaN,07:00:39,NaN,NaN,District Court of Maryland People's Court,8,2019,NaN,NaN
1,B00039,MULTI,ELEC/GENERAL,NaN,210.0,2018-08-02,NaT,2018-08-02,2018-08-02,AFM,NaN,08:14:52,07:48:16,NaN,NaN,Pimlico Academy Public Safety Training Facilit...,8,2019,8.0,2019.0
2,B00120,HEALTH,BUILDING INTERIOR INSPECTION,DELMAR.AUSTIN,18.6,2018-08-02,NaT,2018-08-02,2018-08-02,AFM,NaN,08:38:57,07:51:05,NaN,NaN,Waxter Senior Center,8,2019,8.0,2019.0
3,B00091,HEALTH,PLUMB/OTHER,NaN,0.0,NaT,NaT,NaT,2018-08-02,AFM,NaN,NaN,08:18:21,NaN,NaN,Baltimore Animal Rescue and Care Shelter,8,2019,NaN,NaN
4,B00091,HEALTH,HVAC,NaN,360.0,NaT,NaT,2018-08-02,2018-08-02,AFM,NaN,14:53:42,08:30:37,NaN,NaN,Baltimore Animal Rescue and Care Shelter,8,2019,8.0,2019.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78564 entries, 0 to 78563
Data columns (total 20 columns):
bl_id                        78564 non-null object
site_id                      76540 non-null object
prob_type                    78564 non-null object
completed_by                 28565 non-null object
cost_total                   78564 non-null float64
date_assigned                68610 non-null datetime64[ns]
date_closed                  66739 non-null datetime64[ns]
date_completed               73110 non-null datetime64[ns]
date_requested               78564 non-null datetime64[ns]
manager                      76616 non-null object
time_assigned                1 non-null object
time_completed               72972 non-null object
time_requested               78564 non-null object
time_stat_chg                0 non-null float64
po_number                    9131 non-null object
name                         78564 non-null object
date_requested_month         78564 non-null int64


In [11]:
# work order duration runs from date requested-date completed